In [1]:
import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D

from keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import swish, relu,selu

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

train_set= ('./Experiments/data_pre/mel_spectrogram_imgs/train')
test_set=('./Experiments/data_pre/mel_spectrogram_imgs/test')
val_set= ('./Experiments/data_pre/mel_spectrogram_imgs/val')

train_datagen = image.ImageDataGenerator(rescale= 1./255)
val_datagen= image.ImageDataGenerator(rescale= 1./255)
test_datagen= image.ImageDataGenerator(rescale= 1./255)


train_generator = train_datagen.flow_from_directory(train_set,batch_size =512 ,class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory(test_set,shuffle=True,batch_size =128 ,class_mode = 'categorical')
validation_generator = test_datagen.flow_from_directory(val_set,shuffle=True,batch_size =128 ,class_mode = 'categorical')
x_train, y_train = next(train_generator)
x_val,y_val= next(validation_generator)
x_test, y_test = next(test_generator)

2024-05-03 17:23:16.282458: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 17:23:16.302163: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 17:23:16.302188: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 17:23:16.302789: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-03 17:23:16.306768: I tensorflow/core/platform/cpu_feature_guar

Found 699 images belonging to 10 classes.
Found 151 images belonging to 10 classes.
Found 149 images belonging to 10 classes.


In [2]:
best_params = {'num_layers': 1, 'layer_size': 409, 'dropout_rate': 0.3736932106048628, 'use_batch_norm': True, 'activation': 'selu', 'learning_rate': 0.00010842262717330161}

In [3]:
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint

num_layers = best_params['num_layers']
layer_size = best_params['layer_size']
dropout_rate = best_params['dropout_rate']
use_batch_norm = best_params['use_batch_norm']
activation = best_params['activation']
learning_rate = best_params['learning_rate']


base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the layers in the base model
for layer in base_model.layers[:-1]: #####   modify the number of layers
    layer.trainable = False

# Create a new model
model = Sequential()

# Add the VGG16 base model to the new model
model.add(base_model)
model.add(Flatten())
model.add(Dense(layer_size, input_shape=(x_train.shape[1],)))
model.add(Activation(activation))
model.add(Dropout(dropout_rate))

for _ in range(num_layers):
    model.add(Dense(layer_size))
    if use_batch_norm:
        model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout_rate))

model.add(Dense(10, activation='softmax'))

# Compile the model with hyperparameters
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with hyperparameters
csv_logger = CSVLogger('./logs/VGG_r1_training.csv')
# Define the checkpoint callback
checkpoint = ModelCheckpoint(filepath='./model_weights/VGG_r1_model_weights.h5', 
                             monitor='val_loss', 
                             save_best_only=True,
                             save_weights_only=True,
                             mode='min',
                             verbose=1)


early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', min_delta=0.0001, restore_best_weights=True)
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=150, callbacks=[early_stopping,checkpoint,csv_logger], verbose=1)


2024-05-03 17:23:24.388288: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 17:23:24.407083: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 17:23:24.407108: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 17:23:24.410511: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 17:23:24.410550: I external/local_xla/xla/stream_executor

Epoch 1/150


2024-05-03 17:23:25.788841: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-05-03 17:23:25.879880: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-03 17:23:28.061516: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-03 17:23:28.534638: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f088821cdc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-03 17:23:28.534667: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-05-03 17:23:28.537843: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1714753408.580579   13347 devic

16/16 [==============================] - ETA: 0s - loss: 3.2459 - accuracy: 0.1035
Epoch 1: val_loss improved from inf to 2.92777, saving model to ./model_weights/VGG_r1_model_weights.h5
16/16 [==============================] - 6s 163ms/step - loss: 3.2459 - accuracy: 0.1035 - val_loss: 2.9278 - val_accuracy: 0.1016
Epoch 2/150
16/16 [==============================] - ETA: 0s - loss: 2.9852 - accuracy: 0.1660
Epoch 2: val_loss did not improve from 2.92777
16/16 [==============================] - 2s 141ms/step - loss: 2.9852 - accuracy: 0.1660 - val_loss: 2.9307 - val_accuracy: 0.1172
Epoch 3/150
16/16 [==============================] - ETA: 0s - loss: 2.6811 - accuracy: 0.2070
Epoch 3: val_loss improved from 2.92777 to 2.59323, saving model to ./model_weights/VGG_r1_model_weights.h5
16/16 [==============================] - 2s 148ms/step - loss: 2.6811 - accuracy: 0.2070 - val_loss: 2.5932 - val_accuracy: 0.2031
Epoch 4/150
16/16 [==============================] - ETA: 0s - loss: 2.5743

In [4]:
import matplotlib.pyplot as plt

# Get the training and validation loss from the history
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Get the training and validation accuracy from the history
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Plot the training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.savefig('./img/VGG_r1_loss_plot.png')

# Plot the training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.savefig('./img/VGG_r1_accuracy_plot.png')



/tmp/ipykernel_6012/1491960095.py:19: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_6012/1491960095.py:30: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [5]:
model.evaluate(x_test, y_test)


4/4 [==============================] - 0s 108ms/step - loss: 1.2634 - accuracy: 0.5938


[1.26343834400177, 0.59375]

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score

classes = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

# Assuming you have the predicted labels and true labels
y_pred = model.predict(x_test).argmax(axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Calculate the F1 score
f1 = f1_score(y_true, y_pred, average='weighted')

# Plot the confusion matrix
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()
plt.savefig('./img/VGG_r1_confusion_matrix.png')
# Print the F1 score
print('F1 Score:', f1)


4/4 [==============================] - 0s 105ms/step
F1 Score: 0.5887786047189861


/tmp/ipykernel_6012/3226880583.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
